In [1]:
import pandas as pd
import numpy as np

In [2]:
import sklearn as sk
from sklearn import tree

In [3]:
ratings = pd.read_csv("ratings.csv")
mean_ratings = ratings.groupby("movieId").rating.mean()

In [4]:
movies = pd.read_csv("movies.csv")
movies = movies.set_index("movieId")
movies['rating'] = mean_ratings
movies.head()

,title,genres,rating
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
3,Grumpier Old Men (1995),Comedy|Romance,3.161017
4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
5,Father of the Bride Part II (1995),Comedy,3.267857


In [5]:
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=False)
movies.year.astype(float)
movies.head()

,title,genres,rating,year
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,1995
2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,1995
3,Grumpier Old Men (1995),Comedy|Romance,3.161017,1995
4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,1995
5,Father of the Bride Part II (1995),Comedy,3.267857,1995


In [6]:
genres = ["Drama", "Comedy", "Thrilles", "Action" ,"Adventure", "Crime"]
for genre in genres:
    movies[genre] = movies.genres.str.contains(genre)
movies['good'] = np.where(movies.rating>3.5, "Good", "Bad")
movies.head(5)

,title,genres,rating,year,Drama,Comedy,Thrilles,Action,Adventure,Crime,good
movieId,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,1995,False,True,False,False,True,False,Good
2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,1995,False,False,False,False,True,False,Bad
3,Grumpier Old Men (1995),Comedy|Romance,3.161017,1995,False,True,False,False,False,False,Bad
4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,1995,True,True,False,False,False,False,Bad
5,Father of the Bride Part II (1995),Comedy,3.267857,1995,False,True,False,False,False,False,Bad


In [7]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=2)
m = clf.fit(movies[genres], movies.good)

In [8]:
predictions = m.predict(movies[genres])
pd.crosstab(movies.good, predictions)

col_0,Bad,Good
good,,
Bad,3443,1913
Good,1784,1985


In [9]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(movies.good, predictions))

[[3443 1913]
 [1784 1985]]


In [10]:
correct = movies.good == predictions
correct.mean()

0.59484931506849315

In [13]:
import pydotplus
from IPython.display import Image
dot_data = tree.export_graphviz(m,
  feature_names=genres, class_names=['bad', 'good'], 
  filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\export.py:386: DeprecationWarning: out_file can be set to None starting from 0.18. This will be the default in 0.20.
  DeprecationWarning)


TypeError: object of type 'NoneType' has no len()